Импорт нужных либ

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from prettytable import PrettyTable
%matplotlib inline

# Лабораторная работа №1. Задание 1. Вариант 25
Решается обыкновенное дифференциальное уравнение
$ U''(x) = f(x, U, U'), x \in [a; b] $
С краевыми (граничными) условиями
$ U(a) = A; $
$ \mu_1U(b) + \nu_1U'(b) = B $
Метод решения уравнения $ \varphi(\alpha) = 0 $ - метод половинного деления
Направление интегрирования: $ a \implies b $

Входные параметры

In [2]:
# Границы отрезка, на котором ищется решение:
a, b = -1, 1
# Число отрезков разбиения
N = 50
# Точность решения нелинейного уравнения
epsilon = 1e-11
# Предельное число итераций
K = 1000
# Начальное значение пристрелочного параметра
alpha = 0
# Константы краевых уравнений
A, B = -2, 1

In [3]:
# Шаг
h = (b-a) / N
flag_IER = True
iterations_count = 0 # Для итераций в МПД

x = Symbol('x')
u = Symbol('u')
w = Symbol('w')
# Задание функции
f_exact_solution = 6 * x
exact_solution = lambdify([x, u , w], f_exact_solution)
# Первая производная
fd_exact_solution = diff(f_exact_solution, x) + diff(f_exact_solution, u) + diff(f_exact_solution, w)
derivative_exact_solution = lambdify([x, u , w], fd_exact_solution)
print(fd_exact_solution)
# Вторая производная
f2d_exact_solution = diff(fd_exact_solution, x) + diff(fd_exact_solution, u) + diff(fd_exact_solution, w)
derivative2_exact_solution = lambdify([x, u , w], f2d_exact_solution)
print(f2d_exact_solution)

6
0


# Всякие функции для работы лабы

In [4]:
# Разные функции для работы программы
def x_i():
        x_array = np.zeros(N + 1)
        for i in range(N + 1):
            x_array[i] = float(a + i * h)
        return x_array

# Функция первоначальной задачи Коши
f1 = lambda x, u, w: w

f2 = derivative2_exact_solution

# Метод Рунге Кутта 2 порядка:
def RungeKut(input_u, input_w, input_alpha):
    input_u[0] = A # u
    input_w[0] = input_alpha # w
    k1 = np.zeros(2)
    k2 = np.zeros(2)

    for i in range(N):
        k1[0] = (h * f1(x_array[i], input_u[i], input_w[i]))
        k1[1] = (h * f2(x_array[i], input_u[i], input_w[i]))

        k2[0] = (h * f1(x_array[i] + h, input_u[i] + k1[0], input_w[i] + k1[1]))
        k2[1] = (h * f2(x_array[i] + h, input_u[i] + k1[0], input_w[i] + k1[1]))

        input_u[i + 1] = (input_u[i] + 0.5 * (k1[0] + k2[0]))
        input_w[i + 1] = (input_w[i] + 0.5 * (k1[1] + k2[1]))
    return input_u, input_w

def fiAlpha(input_alpha, input_u_array, input_w_array):
    # В тмпу хранятся приближенные значения точного решения функции
    # В тмпв знач. первой производной функции
    tmp_u, tmp_w = RungeKut(input_u_array, input_w_array, input_alpha)
    # Значение функции на границе - В, где В - значение производной функции с права
    fi_alpha_value = tmp_u[N] - B
    return fi_alpha_value

def different_sign(lvalue, rvalue):
    return lvalue * rvalue < 0

def find_alpha2_value(alpha1, input_u_array, input_w_array):
    phi_alpha1 = fiAlpha(alpha1, input_u_array, input_w_array)
    delta = 1.0
    alpha_2 = alpha1 + delta
    phi_alpha2 = fiAlpha(alpha_2, input_u_array, input_w_array)
    flag_reversed = False
    while not different_sign(phi_alpha1, phi_alpha2):
        alpha_2 = alpha1 + delta
        phi_alpha2 = fiAlpha(alpha_2, input_u_array, input_w_array)
        if phi_alpha1 < 0:
            if not (phi_alpha2 > phi_alpha1 and flag_reversed):
                delta += 0.2
            elif phi_alpha2 > phi_alpha1 and flag_reversed:
                delta -= 0.2
            elif phi_alpha2 < phi_alpha1:
                delta = delta * (-1)
                flag_reversed = True
        if phi_alpha1 > 0:
            if phi_alpha2 > phi_alpha1:
                delta = delta * (-1)
                flag_reversed = True
            else:
                if not phi_alpha2 < phi_alpha1 and flag_reversed:
                    delta += 0.2
                if phi_alpha2 < phi_alpha1 and flag_reversed:
                    delta -= 0.2
    return alpha_2

# МПД
def bisection(neg_point, pos_point, input_epsilon, input_u_array, input_w_array):
    global flag_IER
    global iterations_count
    iterations_count += 1
    if iterations_count > K:
        print('IER = 1:Количество итераций слишком велико')
        flag_IER = False
        return None
    average_point = (pos_point + neg_point) / 2  # Средняя точка между концами интервала
    if abs(neg_point - pos_point) < input_epsilon:  # Основное условие для нахождения нуля функции
        return average_point
    test_value = fiAlpha(average_point, input_u_array, input_w_array)  # Значение функции в средней точке
    if test_value > 0:
        return bisection(neg_point, average_point, input_epsilon, input_u_array, input_w_array)
    elif test_value < 0:
        return bisection(average_point, pos_point, input_epsilon, input_u_array, input_w_array)
    return average_point

# Код алгоритма

In [5]:
if N < 0 or K <= 0 or epsilon < 0:
    flag_IER = False
    IER = 2
    print('IER', IER, ':Неправильные значения')
while flag_IER:
    x_array = x_i()
    u_array = np.zeros(N + 1)
    w_array = np.zeros(N + 1)
    u_array, w_array = RungeKut(u_array, w_array, alpha)

    alpha2 = find_alpha2_value(alpha, u_array, w_array)

    result = bisection(alpha, alpha2, epsilon, u_array, w_array)
    print('Фи в альфа*', fiAlpha(result, u_array, w_array))

    # Заголовок вывода
    if not flag_IER:
        break

    th = ["x", "y", "deltaU", "y'", "deltaU'"]
    td = []
    for i in np.arange(0, N + 1):
        td.append(x_array[i])
        td.append(exact_solution(x_array[i], 0, 0))
        td.append(abs(exact_solution(x_array[i], 0, 0) - u_array[i]))
        td.append(derivative_exact_solution(x_array[i], 0, 0))
        td.append(abs(derivative_exact_solution(x_array[i], 0, 0) - w_array[i]))
    columns = len(th)
    table = PrettyTable(th)
    td_data = td[:]
    while td_data:
        table.add_row(td_data[:columns])
        td_data = td_data[columns:]


    print('EIR = 0:Безошибочная работа программы')
    print('Количество итераций', iterations_count)
    print('Конечное значение параметра пристрелки', result)

    print(table)

    flag_IER = False

Фи в альфа* -5.8186788720604454e-12
EIR = 0:Безошибочная работа программы
Количество итераций 39
Конечное значение параметра пристрелки 1.4999999999970897
+-----------------------+----------------------+---------------------+----+-------------------+
|           x           |          y           |        deltaU       | y' |      deltaU'      |
+-----------------------+----------------------+---------------------+----+-------------------+
|          -1.0         |         -6.0         |         4.0         | 6  | 4.500000000002911 |
|         -0.96         |        -5.76         |  3.8199999999998835 | 6  | 4.500000000002911 |
|         -0.92         | -5.5200000000000005  |  3.639999999999768  | 6  | 4.500000000002911 |
|         -0.88         |        -5.28         |  3.4599999999996514 | 6  | 4.500000000002911 |
|         -0.84         |        -5.04         |  3.279999999999535  | 6  | 4.500000000002911 |
|          -0.8         |  -4.800000000000001  |  3.099999999999419  | 6  | 4

In [6]:
# y = np.zeros(N + 1)
# deltau = np.zeros(N + 1)
# y_ = np.zeros(N + 1)
# deltau_ = np.zeros(N + 1)
# for i in np.arange(0, N + 1):
#         y[i] = exact_solution(x[i])
#         deltau[i] = abs(exact_solution(x[i]) - u[i])
#         y_[i] = derivative_exact_solution(x[i])
#         deltau_[i] = abs(derivative_exact_solution(x[i]) - v[i])
# plt.figure(figsize=(10,6))
# plt.plot(
#     x, y,
#     x, y_
# )
# plt.legend([
#     'Y',
#     "Y'"
# ])
# plt.grid(true)
# plt.title("Функции $Y$ и $Y'$")
# plt.xlabel("X")
# plt.ylabel("Y")
#
# plt.figure(figsize=(10,6))
# plt.plot(
#     x, deltau
# )
# plt.legend([
#     "$\delta$U"
# ])
# plt.grid(true)
# plt.title("Абсолютная погрешность для $Y$")
# plt.xlabel("X")
# plt.ylabel("Y")
#
# plt.figure(figsize=(10,6))
# plt.plot(
#     x, deltau_
# )
# plt.legend([
#     "$\delta$U'"
# ])
# plt.grid(true)
# plt.title("Абсолютная погрешность для $Y'$")
# plt.xlabel("X")
# plt.ylabel("Y")